# Map-reduce in `load_summarize_chain`

In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
import re
from urllib.request import urlopen
import html
import gradio as gr
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
import tiktoken
from pprint import pprint
from tqdm import tqdm

MAP_TEMPLATE_TXT = """Write a detail summary of this text section in bullet points. 
Use '-' for bullet points and answer only the bullet points.
Text:
{text}

SUMMARY:"""
    
COMBINE_TEMPLATE_TXT = """Combine these summaries into a final summary in bullet points.
Use '-' for bullet points and answer only the bullet points.
Text:
{text}

FINAL SUMMARY:"""

map_prompt_txt = MAP_TEMPLATE_TXT
combine_prompt_txt = COMBINE_TEMPLATE_TXT

/Users/pisek/miniconda3/envs/py3p12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configuration settings

In [2]:
with open("sample-text.txt", "r") as f:
    transcript = f.read()

model = "llama3.2"
base_url = "http://localhost:11434"

chunk_size = 2000 # this is in tokens
overlap_size = 0 # this is in tokens

temperature = 0.5

map_num_predict = 256 # number of tokens to predict, Default: 128, -1 = infinite generation, -2 = fill context
combine_num_predict = 2048

# Helper function

In [3]:
def get_text_splitter(chunk_size: int, overlap_size: int):
    return RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=overlap_size)

def convert_text_to_tokens(text, encoder="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(encoder)
    return enc.encode(text)

def get_larger_context_size(token_count):
    num_ctxes = [1024*i for i in range(1, 100)]
    num_ctx = next(ctx for ctx in num_ctxes if ctx > token_count) # pick the first context size that is greater than the token counts
    return num_ctx

# Prepare docs

In [4]:
def convert_transcript_to_split_docs(transcript, chunk_size, overlap_size):
    docs = [Document(
        page_content=transcript,
        # metadata={"source": url}
    )]
    text_splitter = get_text_splitter(chunk_size=chunk_size, overlap_size=overlap_size)
    split_docs = text_splitter.split_documents(docs)
    return split_docs

# Map Step

In [5]:
def run(transcript, chunk_size, overlap_size, map_prompt_txt, combine_prompt_text):
    split_docs = convert_transcript_to_split_docs(transcript, chunk_size, overlap_size)
    
    map_prompt = PromptTemplate(
        template=map_prompt_txt,
        input_variables=["text"]
    )

    combine_prompt = PromptTemplate(
        template=combine_prompt_text,
        input_variables=["text"]
    )

    map_num_ctx = get_larger_context_size(map_num_predict+chunk_size)
    
    llm_map = ChatOllama(
        model=model,
        base_url=base_url,
        temperature=temperature,
        num_ctx=map_num_ctx,
        num_predict=map_num_predict,
        format='',
        verbose=True
    )
    
    summaries = []

    for i, splic_doc in enumerate(tqdm(split_docs, desc="Mapping...")):
        full_prompt = map_prompt.format_prompt(text=splic_doc.page_content)
        output = llm_map.invoke(full_prompt.text)
        summaries.append(output.content)
    
    combined_summaries = "\n".join(summaries)
    combine_prompt_text = COMBINE_TEMPLATE_TXT

    full_prompt = combine_prompt.format_prompt(text=combined_summaries)

    token_counts = len(convert_text_to_tokens(full_prompt.text))

    
    combine_num_ctx = get_larger_context_size(token_counts+combine_num_predict)

    llm_combine = ChatOllama(
        model=model,
        base_url=base_url,
        temperature=temperature,
        num_ctx=combine_num_ctx,
        num_predict=combine_num_predict,
        format='',
        verbose=True
    )

    output_comb = llm_combine.invoke(full_prompt.text)

    return output_comb.content

In [ ]:
output = run(transcript, chunk_size, overlap_size, map_prompt_txt, combine_prompt_txt)

Mapping...: 100%|██████████| 22/22 [02:31<00:00,  6.89s/it]


'- - A potential exploit in physics that could lead to buffer overflow and rounding errors in floating-point synthetic intelligences is being explored.\n\n- - Andre Capathi, a renowned scientist and educator in AI, explains that neural networks are a mathematical abstraction of the brain, but are not directly inspired by it.\n\n- - Neural networks use a sequence of matrix multiplies (dot products) with nonlinearities to learn and make predictions.\n\n- - Despite being simple mathematically, neural networks can exhibit surprising emergent behaviors, especially when trained on complex problems.\n\n- - GPT-like models are a type of neural network that can be prompted to solve problems and provide remarkably consistent solutions.\n\n- - Researchers believe that intelligent life is not unique to humans, and it\'s possible that there are trillions of intelligent alien civilizations out there.\n\n- - The origin of life on Earth was not as rare or magical as previously thought, but rather a na